<img src="https://i.imgur.com/XSzy00d.png" style="float:right;width:150px">

**Matplotlib und Widgets**

# Einleitung

## Lernziele

* Sie können ein **Line Chart** erstellen
* Sie können einen **Scatterplot** erstellen
* Sie verstehen, wann sie eher ein Line Chart und wann eher ein Scatterplot verwenden
* Sie kennen Möglichkeiten von **Widgets** zur Erstellung von GUIs
* Sie können auf **Änderungen** in **Widgets** reagieren

# Matplotlib

[Matplotlib](https://matplotlib.org/) ist ein Modul, welches zur Visualisierung von Daten dient. Matplotlib harmoniert insbesondere mit den Modulen NumPy und Pandas aus [Lektion 10](../Lektion_10/NumPy_Pandas.ipynb) sehr gut. Matplotlib kann sowohl NumPy Arrays als auch Pandas DataFrames visualisieren.

## Grundlegender Aufbau

Matplotlib kann man auf sehr unterschiedliche Arten und Weisen benutzen, das macht dass Nachvollziehen von Beispielen aus dem Internet zuweilen etwas mühsam. Die einfachste Art und Weise funktioniert folgendermassen:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.plot([64, 32, 16, 8, 4, 2, 1])

plt.show()

Zuerst muss das [Magic Command](https://ipython.readthedocs.io/en/stable/interactive/magics.html#built-in-magic-commands) `%matplotlib inline` ausgeführt werden, damit die Visualisierungen innerhalb der Zelle wiedergegeben werden. Danach wird das Modul `matplotlib.pyplot` unter dem Namen `plt` importiert. Dies ist eine weitverbreitete Konvention. Die Angabe, was denn überhaupt geplottet werden soll wird mit dem Befehl `plt.plot()` gemacht. Dieser Funktion kann eine Liste mit Zahlen übergeben werden. Falls nur eine Liste übergeben wird, interpretiert Matplotlib die Zahlen so, dass sie als y-Werte entlang der x-Achse beginnend mit x = 0 gezeichnet werden. Mit `plt.show()` kann dann schlussendlich der Plot auch angezeigt werden.

Werden zwei Listen der Funktion `plt.plot()` übergeben, interpretiert Matplotlib diese als x- und y-Werte:

In [ ]:
plt.plot([10, 20, 30, 40, 50, 60, 70], [1, 2, 4, 8, 2, 1, -3])
plt.show()

<div class="gk-exercise">


Erstelle einen Plot, welcher als y-Werte 100 Zufallszahlen zwischen 0 und 100 darstellt. Führe den Quellcode mehrere male aus, um zu prüfen, ob sich die Zufallszahlen und der Plot dementsprechend tatsächlich ändern.

<details>
<summary>Tipp</summary>
    <p>Hinweis: Es gibt verschiedene Möglichkeiten, eine Liste von Zufallszahlen zu erstellen. Nimm am besten die Funktion <code>random.randint()</code> aus dem Modul NumPy, welches es ermöglicht, nicht nur einzelne Zufallszahlen, sondern ganze Listen davon zu erstellen. Eine Beschreibung zu <code>random.randint()</code> findest du <a href="https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html">hier</a>.</p>
</details>
</div>

In [ ]:
# YOUR CODE HERE

## Import von interessanteren Daten

Um möglichst schnell interessantere Daten zu visualisieren, werden nachfolgend die Zeitreihendaten aus dem Hitzeinsel-Monitoring verwendet, welches wir bereits in der letzten [Lektion](../Lektion_11/GIS.ipynb) kennengelernt haben.

In der letzten Lektion haben wir den `latest`-Endpoint verwendet, um eine Liste aller Sensoren und deren aktuellsten Messwerte zu erhalten. In dieser Lektion nutzen wir den `timeseries`-Endpoint, um die historischen Messwerte eines spezifischen Sensors auszuwerten. Dazu muss über URL-Parameter die `stationId` des entsprechenden Sensors angegeben werden. Optional können mit den Parametern `date_from` und `date_to` der gewünschte Zeitraum definiert werden.

Nachfolgend der Link des Endpoints mit den URL-Parametern:  
`https://smart-urban-heat-map.ch/api/v2/timeseries?stationId=STATION_ID&date_from=DATE_FROM&date_to=DATE_TO`

Die grossgeschriebenen Platzhalter (`STATION_ID`, `DATE_FROM`, `DATE_TO`) müssen jeweils vor dem Aufruf angepasst werden.

Bevor wir die ersten Messdaten eines Sensors abrufen können, müssen wir zuerst die verfügbaren Sensoren laden. Dazu verwenden wir den `latest`-Endpoint aus der letzten Lektion und eine Dict-Comprehension, um ein Dictionary zu erstellen, bei dem der `name` der Sensoren als Schlüssel und die `stationId` als Wert verwendet wird.

In [ ]:
import requests

result = requests.get("https://smart-urban-heat-map.ch/api/v2/latest").json()
sensors = {
            feature["properties"]["name"]: feature["properties"]["stationId"] for feature in result["features"] 
                if feature["properties"]["outdated"] == False and feature["properties"]["measurementsPlausible"] == True
        }

display(sensors)

Die `stationId` kann nun aus dem Sensors-Dictionary ausgelesen und für den Aufruf des `timeseries`-Endpoints verwendet werden.

In [ ]:
station_id_zollikofen = sensors["Zollikofen 3m"]
station_id_zytglogge = sensors["Zytglogge"] 
time_from = "2025-08-10T00:00:00Z"
time_to = "2025-08-13T00:00:00Z"

result_zollikofen = requests.get(f"https://smart-urban-heat-map.ch/api/v2/timeseries?stationId={station_id_zollikofen}&timeFrom={time_from}&timeTo={time_to}").json()
result_zytglogge = requests.get(f"https://smart-urban-heat-map.ch/api/v2/timeseries?stationId={station_id_zytglogge}&timeFrom={time_from}&timeTo={time_to}").json()

display(result_zollikofen)

Als Resultat wird die `stationId` und die Messwerte als Liste in den `values` zückgegeben. Jeder Messwert beinhaltet einen Zeitstempel `dateObserved`, `temperature` und `relativeHumidity`. Zur einfacheren Verarbeitung können die Daten in ein pandas DataFrame geladen werden.


In [ ]:
import pandas as pd

sensor_zollikofen_df = pd.DataFrame(result_zollikofen["values"])
sensor_zytglogge_df = pd.DataFrame(result_zytglogge["values"])

display(sensor_zollikofen_df.head(5))

## Line Chart

Die wohl fundamentalste Visualisierung ist das Liniendiagramm (Line Chart). Ein Liniendiagramm ist besonders sinnvoll, wenn bestimmte Beobachtungen entlang einer kontinuierlichen Grösse variieren. Ein typisches Beispiel dafür ist die Zeitreihenanalyse (Time Series), bei der eine bestimmte Grösse entlang einer Zeitachse untersucht wird. 

Eine solche Zeitreihenanalyse soll nun mit den Temperaturdaten durchgeführt werden:

In [ ]:
plt.plot(sensor_zollikofen_df["temperature"])

plt.show()

In [ ]:
type(sensor_zollikofen_df["temperature"])

Matplotlib arbeitet gut mit Objekten aus NumPy und Pandas zusammen, deshalb kann hier der Datentyp der `Series` sofort geplottet werden.

Wenig hilfreich ist noch die Angabe auf der x-Achse, welches die Zeilenzahl des ursprünglichen DataFrames darstellt. Aber hier soll natürlich das Datum angezeigt werden:

In [ ]:
plt.plot(sensor_zollikofen_df["dateObserved"], sensor_zollikofen_df["temperature"])
plt.show()

Das dauert ziemlich lange und schlussendlich ist die Ausgabe nicht besonders hilfreich. Der Grund dafür ist, dass die Werte von 'Datum' für Menschen natürlich sofort als Datum erkennbar sind, der Computer weiss zuerst aber mal nicht, dass er dies als Datumswerte interpretieren soll. Das muss dem Computer zuerst mitgeteilt werden:

In [ ]:
sensor_zollikofen_df["dateObserved"] = pd.to_datetime(sensor_zollikofen_df["dateObserved"])
sensor_zollikofen_df.info()
sensor_zollikofen_df.head(5)

Mit der Funktion `to_datetime()` können die bisherigen Strings in "echte" Zeitdaten verwandelt werden. Die Funktion `info()` liefert die Datentypen für jede Spalte eines DataFrames, wo jetzt erkennbar ist, dass die Spalte 'datum' nun ein `datetime64` Objekt ist. Bei der Darstellung über `head()` ist der Unterschied allerdings nicht sichtbar. Wenn jetzt nochmals der Plot gezeichnet wird, passt die Beschriftung der x-Achse:

In [ ]:
plt.plot(sensor_zollikofen_df["dateObserved"], sensor_zollikofen_df["temperature"])
plt.show()

Jetzt soll ein Plot erstellt werden, bei dem gleichzeitig die Temperaturen aus **Zollikofen** und vom **Zytgloggen** sichtbar sind.

In [ ]:
sensor_zytglogge_df["dateObserved"] = pd.to_datetime(sensor_zytglogge_df["dateObserved"])

plt.plot(sensor_zollikofen_df["dateObserved"], sensor_zollikofen_df["temperature"], sensor_zytglogge_df["dateObserved"], sensor_zytglogge_df["temperature"])
plt.show()

Es ist also möglich, bei der Funktion `plot()` einfach weitere x/y Datenpaare anzugeben, die dann übereinandergelegt gezeichnet werden.

Nun soll der Plot mit weiteren Zusatzangaben verfeinert werden:

In [ ]:
import matplotlib.dates as mdates

plt.plot(sensor_zollikofen_df["dateObserved"], sensor_zollikofen_df["temperature"], sensor_zytglogge_df["dateObserved"], sensor_zytglogge_df["temperature"])
plt.suptitle("Temperaturen in Zollikofen und beim Zytglogge")
plt.xlabel("Zeit")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%y %H:%M'))

plt.ylabel("Temperatur [°C]")
plt.legend(["Zollikofen", "Zytglogge"])
plt.grid(color = "grey", linestyle = "--", linewidth=1)
plt.show()

Die Grösse der ausgegebenen Grafik lässt sich über `plt.rcParams['figure.figsize'] = [x, y]` anpassen:

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.plot(sensor_zollikofen_df["dateObserved"], sensor_zollikofen_df["temperature"], sensor_zytglogge_df["dateObserved"], sensor_zytglogge_df["temperature"])
plt.suptitle("Temperaturen in Zollikofen und beim Zytglogge")
plt.xlabel("Zeit")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%y %H:%M'))
plt.ylabel("Temperatur [°C]")
plt.legend(["Zollikofen", "Zytglogge"])
plt.grid(color = "grey", linestyle = "--", linewidth=1)
plt.show()

Die Grafiken können einfach auf dem Computer gespeichert werden, indem mit 'Shift + Rechtsklick' auf die Grafik geklickt und dann 'Grafik speichern unter...' gewählt wird.

<div class="gk-exercise">

Erstelle für den Sensor beim Zytglogge ein neues Diagramm mit der Temperaturkurve und markiertem Maximum, Minimum und Mittelwert. Für die Markierung kann mit `plt.axhline(Wert, label='Beschriftung')` eine horizontale Linie auf der Höhe von Wert gezeichnet werden. Achte darauf, dass alle Achsen und Linien gut beschriftet sind und das Diagramm ansprechend formatiert wird.

<details>
    <summary>Tipps</summary>
    <p>Die statistischen Werte können mittels <i>df["Spaltenname"].max()</i> etc. abgefragt werden</p>
    <details>
    <summary>Tipp</summary>
    <p>Mit <i>color='green'</i> kann die Farbe der Linie angepasst werden.</p>
</details>
</details>
</div>

In [ ]:
# YOUR CODE HERE


***

## Scatterplot

Das **Streudiagramm** ist ein weiterer wichtiger Visualisierungstyp. Es kommt dann zum Einsatz, wenn ein allfälliger Zusammenhang (Korrelation) zwischen zwei Variablen einer Beobachtung untersucht werden soll. Beim Streudiagramm werden Wertepaare in einem x-y Koordinatensystem abgebildet. Die Werte der Variablen stehen dabei nicht in einem kontinuerlichen Zusammenhang (im Gegensatz bspw. zu einer Zeitreihe, wo die verschiedenen x-Werte (Zeiten) in einer Abfolge stehen und es deshalb Sinn macht, eine verbindende Linie zu zeichnen).

Nachfolgend sollen Daten zum Geysir "[Old Faithful](https://de.wikipedia.org/wiki/Old_Faithful)" im Yellowstone National Park untersucht werden. Eine CSV Datei ist unter [data/old_faithful.csv](data/old_faithful.csv) verfügbar. Diese beinhaltet zwei Spalten. In der einen Spalte ('duration') wird die Dauer eines Ausbruchs in Minuten angegeben, in der zweiten Spalte ('waiting') die anschliessend erfolgte Wartezeit in Minuten bis zum nächsten Ausbruch.

Die Frage ist nun, ob diese zwei Grössen in irgendeinem Zusammenhang stehen. Dies kann versucht werden, indem die zwei Spalten angezeigt werden:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

old_faithful = pd.read_csv("data/old_faithful.csv", sep = ";")

display(old_faithful.head(50))

Es ist ziemlich schwierig, eine so grosse Anzahl Zahlen auf einen Blick zu erfassen und Muster zu erkennen. Viel einfacher geht es mit einer Visualisierung:

In [ ]:
plt.scatter(old_faithful["duration"], old_faithful["waiting"])
plt.show()

Dank diesem Streuplot wird ersichtlich, dass es zwei "Cluster" gibt, die sich dadurch auszeichnen, dass bei einem kurzen Ausbruch tendenziell eine kurze Wartezeit folgt, während nach einem langen Ausbruch auch eine lange Wartezeit folgt.

Sollen bei einem Streudiagramm mehrere Datensätze gleichzeitig in einem Diagramm abgebildet werden, kann die `plt.scatter()` Funktion einfach mehrmals mit unterschiedlichen Daten aufgerufen werden. Innerhalb der gleichen Zelle werden dann alle Daten in einem Diagramm dargestellt. Somit können wir den Scatterplot noch etwas übersichtlicher darstellen, in dem wir die beiden "Cluster" unterschiedlich einfärben:

In [ ]:
old_faithful_short = old_faithful[old_faithful.duration <= 3.25]
old_faithful_long = old_faithful[old_faithful.duration > 3.25]

plt.rcParams['figure.figsize'] = [12, 8]

plt.scatter(old_faithful_short["duration"], old_faithful_short["waiting"])
plt.scatter(old_faithful_long["duration"], old_faithful_long["waiting"])
plt.suptitle("Old Faithful Geysir")
plt.xlabel("Ausbruchsdauer [min]")
plt.ylabel("Wartezeit nach Ausbruch [min]")
plt.grid(color = "grey", linestyle = "--", linewidth=1)

plt.show()

# Widgets

Widgets sind kleine grafische Elemente, welche die Ein- und Ausgabe vereinfachen. Damit dienen sie dazu, eine Art **Grafical User Interface** (GUI) zu erstellen. Um mit Widgets zu arbeiten, muss das entsprechende Modul `ipywidgets` importiert werden:

In [ ]:
import ipywidgets as widgets

Typischerweise werden die Widgets einer Variable zugewiesen, damit man sie anzeigen, auslesen und verändern kann:

In [ ]:
w = widgets.IntSlider()

Mit der Funktion `display()` können die Widgets im Output angezeigt werden. Die Zelle muss also immer ausgeführt werden, damit das Widget angezeigt wird.

In [ ]:
display(w)

Der Wert, der mit dem Schieberegler eingestellt wird, kann mit der Eigenschaft `value` des Widget Objekts ausgelesen werden:

In [ ]:
print("Der Wert des Reglers ist:" + str(w.value))

## Reagieren auf Änderungen von Widgets

Soll beispielsweise beim Drücken eines Buttons sofort eine Aktion ausgelöst werden (ohne dass dafür das Ausführen einer weiteren Zelle notwendig ist), ist ein etwas aufwändigeres Vorgehen nötig: 

In [ ]:
button = widgets.Button(description="Klick mich!")
output = widgets.Output()

display(button, output)

def button_clicked(b):
    with output:
        print("Der Knopf '" + b.description +  "' wurde gedrückt")

button.on_click(button_clicked)

Zuerst wird ein Button und ein spezielles Output Widget erzeugt, diese werden dann beide über `display()` angezeigt, wobei vom Output vorerst noch nichts zu sehen ist. Dann wird eine Funktion `button_clicked(b)` definiert, welche etwas tun soll, sobald der Button gedrückt wird. Anschliessend wird mit `button.on_click(button_clicked)` dem Button bei einem Klick diese Funktion zugewiesen.

Bei dieser Konstruktion wird der Funktion, die bei einem Klick aufgerufen wird, also der Funktion `button_clicked()` automatisch ein Argument übergeben, das Informationen über den gedrückten Knopf enthält, die Funktion muss also über einen Parameter verfügen, um dieses Argument aufzunehmen. Im Falle des Buttons kann auf die Eigenschaft `description` des gedrückten Buttons zugegriffen werden, das die Beschriftung des Buttons enthält.

Da es mit dieser Konstruktion nicht möglich ist, per einfachem `print()` Befehl einen Output zu erzeugen, brauchen wir das Output Widget, welches innerhalb der Funktion mit Hilfe des Schlüsselwortes `with` in einem sogenannten *Context Manager* (genau wie beim Arbeiten mit Dateien) aufgerufen wird.

Das Dropdown Widget ist eine weitere interessante Möglichkeit:

In [ ]:
dropdown = widgets.Dropdown(
    options = ["Bern", "Biel", "Thun"],
    description = "Stadt")

display(dropdown)

Um auf die Wahl aus dem Dropdown Widget zu reagieren, kann die oben verwendete Methodik, leicht angepasst, angewandt werden:

In [ ]:
dropdown = widgets.Dropdown(
    options = ["Bern", "Biel", "Thun"],
    description = "Stadt")

output = widgets.Output()

display(dropdown, output)

def on_value_change(change):
    with output:
        print(change)
        
dropdown.observe(on_value_change, names = "value")

Um den Wechsel des Dropdown Menus zu überwachen, können wir die Methode `observe` anwenden. Diese braucht zwei Argumente, nämlich die Funktion, die bei einem Wechsel ausgelöst werden soll (hier `on_value_change()` und als zweites Argument, was genau überwacht werden soll. Das ist für ein Dropdown Widget typischerweise die Werte, also `names = 'value'`. Das Parameter Objekt, das beim Wechsel des Dropdown Widgets der Funktion `on_value_change` übergeben wird, ist ein Dictionary, das verschiedene Schlüssel-Wert Paare beinhaltet, der interessante Wert dabei ist `new` für den neu ausgewählten Eintrag.

## Weitere Infos zu Widgets

Eine gute Dokumentation zu den Widgets ist unter [https://ipywidgets.readthedocs.io](https://ipywidgets.readthedocs.io/en/stable/index.html) verfügbar.

# Schlussaufgabe

<div class="gk-exercise">
Erstelle eine interaktive Visualisierung für die Zeitreihen der Hitzeinsel-Monitoring-Sensoren. Dabei soll zuerst mit Hilfe eines Dropdown-Widgets ein Sensor ausgewählt werden, von dem anschliessend die Temperatur- und relativen Luftfeuchtigkeitswerte vom `timeseries`-Endpoint geladen und in einem Diagramm angezeigt werden. Die möglichen Sensoren im Dropdown-Widget sollen dabei aus den Daten des `latest`-Endpoints geladen werden.  

Bonusaufgabe: Verwende zusätzlich zwei Widgets, um `time_from` und `time_to` auszuwählen.  

<details>
    <summary>Tipps</summary>
    <p>Am besten werden zwei y-Achsen erzeugt (je eine links und eine rechts), da die Temperaturen und die Luftfeuchtigkeiten unterschiedliche Wertebereiche haben. Dies kann mit Hilfe von <code>plt.twinx()</code> umgesetzt werden.</p>
    <p>Für die Bonusaufgabe kann das Widget `widgets.DatetimePicker` verwendet werden.</p>
</details>
</div>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets

# YOUR CODE HERE

# Zusammenfassung

In diesem Notebook wurde aufgezeigt, wie aus Daten Visualisierungen erstellt werden können. Damit wurde auch aufgezeigt, dass gewisse Strukturen in Daten mit Hilfe einer Visualisierung sehr viel schneller erkennbar sind als beim Blick auf die Rohdaten.

Die beiden wichtigen Typen Line Chart und Scatterplot wurden eingeführt und es ist aufgezeigt worden, wann diese zum Einsatz kommen können.

Schliesslich wurde die Möglichkeit der Widgets aufgezeigt. Diese eignen sich gut, um Eingaben von der Benutzerin abzuholen oder als Möglichkeit zur Ausgabe eines Resultates aus dem Programmablauf.

# Impressum

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg" /></a><br />Dieses Werk ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Namensnennung - Weitergabe unter gleichen Bedingungen 4.0 International Lizenz</a>.

Autoren: [Jakob Schärer](mailto:jakob.schaerer@unibe.ch), [Lionel Stürmer](mailto:lionel.stuermer@bfh.ch) <br>
Ursprünglicher Text von: Noe Thalheim, Benedikt Hitz-Gamper


## Credits


* [Old Faithful Daten](https://www.stat.cmu.edu/~larry/all-of-statistics/=data/faithful.dat)
* [Smart Urban Heat Map](https://smart-urban-heat-map.ch)
```
There are 10 types of people in the world.
Those who understand binary, and those who don't.
```